## Sphere 1 and 2

In [1]:
import numpy as np
import pandas as pd
import bats
import scipy.spatial.distance as distance

In [2]:
# generate circle (unnoisy)
def gen_circle(n = 50, d = 2):
    X = np.random.normal(size=(n,d))
    # X = np.random.uniform(-1,1,(n,2))
    X = X / np.linalg.norm(X, axis=1).reshape(-1,1)
    # std = 0.1
    # Y = Y + np.random.normal(size=(n,2), scale = 0.1 )
    return X

n = 200
d = 2
sigma = 0.001
# create dataset
X = gen_circle(n, d) # unnoisy circle
Y = X + np.random.normal(size=(n,d), scale = sigma )

In [3]:
dmax = 2 # max dimension of filtration, i.e., compute up to H_{d_max-1}
def build_filt_find_update_info(X, Y, dmax):
    # build filtration
    DX = distance.squareform(distance.pdist(X))
    rX = bats.enclosing_radius(bats.Matrix(DX))
    R0 = bats.LightRipsFiltration(bats.Matrix(DX), rX, dmax)

    # build filtration
    DY = distance.squareform(distance.pdist(Y))
    rY = bats.enclosing_radius(bats.Matrix(DY))
    R1 = bats.LightRipsFiltration(bats.Matrix(DY), rY, dmax)

    U = bats.UpdateInfo2(R0, R1)
    return R0, U

In [4]:
def update_scale(U,X):
    '''
    Compute
    (1) normalized kendall-tau distance 
    (2) number simplices added divided by original number of simplices 
    (3) number simplices deleted divided by the original number of simplices.
    '''
    
    k = 0
    maxk = 0
    total_ncells = 0
    nadditions = 0
    for d in range(X.maxdim() + 1):
        nd = X.ncells(d)
        maxk += (nd * (nd - 1)) // 2
        k += bats.kendall_tau(U.perm[d])
        total_ncells += nd
        nadditions += len(U.insertion_cols[d])
        
    return k / maxk,  nadditions/total_ncells, sum(U.ndeletions)/total_ncells

R0, U = build_filt_find_update_info(X,Y, dmax =2)
update_scale(U, R0)

(0.004725804682776271, 0.0016843002871447204, 0.03188453674012223)

In [5]:
n = 200
d = 3
sigma = 0.001
# create dataset
X = gen_circle(n, d) # unnoisy circle
Y = X + np.random.normal(size=(n,d), scale = sigma )

R0, U = build_filt_find_update_info(X,Y, dmax =3)
update_scale(U, R0)

(0.004448693654552695, 0.0009444554916117498, 0.012261239976302568)

## Klein3

In [13]:
dmax_complex = 3 # maximum dimension of complex, will compute PH until one dimension lower
n_sample = 100 # number of sample points
df = pd.read_csv('klein_bottle_pointcloud_new_400.txt', delimiter = " ", header = None)
df = df.iloc[:,:-1] # the last column is NaN, we need it
    
X = np.array(df) # transform into numpy array
# Uniform randomly generate points
n_total_rows = X.shape[0]
random_indices = np.random.choice(n_total_rows, size=n_sample, replace=False)
X = X[random_indices, :]
Y = X + np.random.normal(size=(X.shape[0], X.shape[1]), scale = 0.001 )

In [14]:
R0, U = build_filt_find_update_info(X, Y, dmax =3)
update_scale(U, R0)

(0.0009638290848562882, 0.0, 0.0)

## Dragon

In [15]:
n_sample = 400 # number of sample points
df = pd.read_csv('dragon_vrip.ply.txt_1000_.txt', delimiter = " ", header = None)
dmax_complex = 2
X = np.array(df) # transform into numpy array
n_total_rows = X.shape[0]
random_indices = np.random.choice(n_total_rows, size=n_sample, replace=False)
X = X[random_indices, :]
X2 = X + X * 0.01 * np.random.randn(X.shape[0],X.shape[1])

R0, U = build_filt_find_update_info(X, X2, dmax =dmax_complex)
update_scale(U, R0)

(0.022069424134177233, 0.021481652573601368, 0.007133504477299316)

## Bunny

In [17]:
from plyfile import PlyData
def read_ply_to_numpy_arr(file_path):

    plydata = PlyData.read(file_path) # read file
    data = plydata.elements[0].data # read data
    data_pd = pd.DataFrame(data) # Convert to DataFrame, because DataFrame can parse structured data
    data_np = np.zeros(data_pd.shape, dtype=np.float32) # Initialize the array of stored data
    property_names = data[0].dtype.names # read the name of the property
    for i, name in enumerate(property_names): # Read data according to property, so as to ensure that the data read is the same data type.
        data_np[:, i] = data_pd[name]

    return data_np[:,:3] # only use the first 3 columns

In [20]:
n_sample = 400 # number of sample points 
full_X = read_ply_to_numpy_arr('bun000.ply')
n_total_rows = full_X.shape[0]
random_indices = np.random.choice(n_total_rows, size=n_sample, replace=False)
X = full_X[random_indices, :]
X2 = X + X * 0.01 * np.random.randn(X.shape[0],X.shape[1])

dmax_complex = 2
R0, U = build_filt_find_update_info(X, X2, dmax =dmax_complex)
update_scale(U, R0)

(0.014960165865476565, 0.0009324730703427441, 0.017011479265594272)

## H3N2

In [21]:
dmax_complex = 2 # maximum dimension of complex, will compute PH until one dimension lower
n_sample = 400 # number of sample points
# create dataset
df = pd.read_csv('H3N2.all.nt.concat.fa_hdm.txt_point_cloud.txt', delimiter = " ", header = None)
df = df.iloc[:,:-1] # the last column is NaN
    
X = np.array(df) # transform into numpy array
# Uniform randomly generate points
n_total_rows = X.shape[0]
random_indices = np.random.choice(n_total_rows, size=n_sample, replace=False)
X = X[random_indices, :]
X2 = X + X * 0.01 * np.random.randn(X.shape[0],X.shape[1])

R0, U = build_filt_find_update_info(X, X2, dmax =dmax_complex)
update_scale(U, R0)

(0.0058003253789867286, 0.0002797367140582588, 0.0007457346187348404)